<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/simulate_and_analyze_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
simulate_and_analyze.py

- Simulates log_output & log_beyond_field (with optional noise)
- Computes summary statistics and regression slope
- Generates 2D/3D visualizations
- Saves data and plots to disk
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import Axes3D

def simulate(
    total_steps: int,
    init_log_output: float,
    init_log_beyond: float,
    growth_per_step: float,
    breach_step: int,
    breach_value: float
) -> pd.DataFrame:
    """Generate basic log_output & log_beyond_field series."""
    steps = np.arange(1, total_steps + 1)
    log_out = init_log_output + growth_per_step * (steps - 1)
    log_bey = np.full(total_steps, init_log_beyond, dtype=float)
    idx = breach_step - 1
    if 0 <= idx < total_steps:
        log_bey[idx:] = breach_value
    return pd.DataFrame({
        'step': steps,
        'log_output': log_out,
        'log_beyond_field': log_bey
    })

def simulate_with_noise(
    base_df: pd.DataFrame,
    noise_sd: float
) -> pd.DataFrame:
    """Add Gaussian noise to log_output."""
    df = base_df.copy()
    df['log_output_noisy'] = df['log_output'] + np.random.normal(0, noise_sd, size=len(df))
    return df

def summary_statistics(df: pd.DataFrame, col: str) -> dict:
    """Return mean, variance, min, and max for a column."""
    series = df[col]
    return {
        'mean': float(series.mean()),
        'variance': float(series.var()),
        'min': float(series.min()),
        'max': float(series.max())
    }

def fit_regression_slope(df: pd.DataFrame, x_col: str, y_col: str) -> float:
    """
    Fit y = m*x + b and return slope m.
    Uses ordinary least squares.
    """
    X = df[[x_col]].values
    y = df[y_col].values
    model = LinearRegression().fit(X, y)
    return float(model.coef_[0])

def plot_2d(df: pd.DataFrame, filename: str):
    """2D plot of log_output and log_beyond_field."""
    plt.figure(figsize=(8, 5))
    plt.plot(df['step'], df['log_output'], label='log_output')
    plt.plot(df['step'], df['log_beyond_field'], label='log_beyond_field')
    plt.xlabel('Step')
    plt.ylabel('Log Value')
    plt.legend()
    plt.tight_layout()
    plt.savefig(filename, dpi=150)
    plt.close()

def plot_3d_noisy(df: pd.DataFrame, filename: str):
    """3D scatter: step vs log_output vs noisy log_output."""
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(df['step'], df['log_output'], df['log_output_noisy'], c='teal', s=2)
    ax.set_xlabel('Step')
    ax.set_ylabel('log_output')
    ax.set_zlabel('log_output_noisy')
    plt.tight_layout()
    plt.savefig(filename, dpi=150)
    plt.close()

def main():
    # Parameters
    TOTAL_STEPS     = 200_000
    INIT_LOG_OUT    = 230.2585
    INIT_LOG_BEY    = 0.0
    GROWTH_PER_STEP = 0.00005
    BREACH_STEP     = 100_000
    BREACH_VALUE    = 0.0001
    NOISE_SD        = 0.02   # Standard deviation for log_output noise

    # 1) Simulate base and noisy data
    df_base = simulate(TOTAL_STEPS, INIT_LOG_OUT, INIT_LOG_BEY,
                       GROWTH_PER_STEP, BREACH_STEP, BREACH_VALUE)
    df_noisy = simulate_with_noise(df_base, NOISE_SD)

    # 2) Persist to CSV
    df_noisy.to_csv('logs_noisy.csv', index=False)
    print("Saved logs_noisy.csv")

    # 3) Summary statistics
    stats = summary_statistics(df_noisy, 'log_output_noisy')
    print("Noisy log_output stats:", stats)

    # 4) Regression slope of log_output_noisy vs step
    slope = fit_regression_slope(df_noisy, 'step', 'log_output_noisy')
    print(f"Estimated slope: {slope:.6e}")

    # 5) Generate plots
    plot_2d(df_base, 'plot_2d.png')
    plot_2d(df_noisy, 'plot_2d_noisy.png')
    plot_3d_noisy(df_noisy, 'plot_3d_noisy.png')
    print("Plots generated: plot_2d(.png), plot_2d_noisy.png, plot_3d_noisy.png")

if __name__ == '__main__':
    main()